## Analyzing borrowers’ risk of defaulting

The goal of this project was to prepare a report for a bank’s loan division. Identify factors which influence the borrowers ability to repay/default on the loan. Following factors were studies:
1. Kids and no kids
2. Married + living together and single
3. Income groups
4. Loan purpose

And historical data on customers’ credit worthiness was used to understand and create a customer profile which were more likely to default on the loan

Based on the findings of the project a  **credit scoring** of a potential customer would be created which would be used to evaluate the ability of a potential borrower to repay their loan.

## Step 1. Open the data file and exploring the data

import pandas as pd
data = pd.read_csv("https://code.s3.yandex.net/datasets/credit_scoring_eng.csv")

data




In [1]:
import pandas as pd
credit_scoring = pd.read_csv("https://code.s3.yandex.net/datasets/credit_scoring_eng.csv")

credit_scoring.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


In [2]:
credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
credit_scoring.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


In [4]:
credit_scoring.duplicated().sum()

54

Data quality findings:
- There are erroneous values , like negative days of employment, negative number of children, ,400K days employed and 0 age.
- There are also missing values in days employed and total income, which maybe because these data points are interconnected through calculations
- 54 records are duplicated

## Step 2. Data preprocessing

In this section we would be addressing the following issues: 
1. Replacing the missing values with a suitable method,
2. Studying and replacing  suspicious data points wherever possible
3. Standardising the text
4. Identifying and addressing dupicate values
5. Changing the data types based on the variable type

Strategies used for cleaning the data:

1. Missing values :
    a. Total_income - we will replace the total income by grouping the data by income type and replacing the total income based on the income type
    b. Days_employed - Converting all the negative values to positive values, addressing the outliers and then replacing the missing values with the mean/median values
  
2. Erroneous values : 
    a. Negative days of employment - changing them to positive
    b. Negative values in number of children - replacing them with the nearest positive number
    c. 20 in the number of children - replacing it with 2 as it seems extra 0 was added while data entry 
    d. Gender XNA - keeping is as is, because maybe the customer did not want to declare their gender. And there is no reliance on this data. So I am not doing anything about it 
    e. 0 years in the age column - was not addressed since it was not a variable of interest  
    
3. Duplicated values :
    a. Obvious = 54
    b. However in the education type the same education type , is represented in 2 different ways smaller letters and capital letters, this was standardized

4. Changing data types to match the variable type :
    a.  The data type for days employed and total income was chnaged from float to a whole number (int)
</div>

In [5]:
# Replacing the null values in total_income by median of data grouped by income type
# Assuming that the total income is dependent on the income type, the missing values were replaced by 
# grouping the data by income type and replacing the missing value by the median of the group
credit_scoring['total_income'] = credit_scoring.groupby('income_type')['total_income'].apply(lambda x:x.fillna(x.median()))

In [6]:
#replacing the negative values in days_employed by positive_values 
credit_scoring['days_employed_positive'] = credit_scoring['days_employed'].abs()

In [7]:
# checking for extremely large values in days_employed
days_employed_filter = credit_scoring['days_employed_positive'] >= 18250

In [8]:
# Addressing these outliers in the days_employed
# dividing them by 24, assuming instead of years , hours of employment were recorded
credit_scoring.loc[credit_scoring['days_employed_positive'] >= 18250, 'days_employed_positive'] = credit_scoring['days_employed_positive']/24

In [9]:
# Finally replacing the missing values in days_employed
# Assuming that the number of days employed is dependent on the income type, the missing values were replaced by 
# grouping the data by income type and replacing the missing value by the median of the group
credit_scoring['days_employed_positive'] = credit_scoring.groupby('income_type')['days_employed_positive'].apply(lambda x:x.fillna(x.median()))

In [10]:
# Addresing the erroneous values in number of children

credit_scoring.loc[credit_scoring['children'] == -1,'children']= 1
credit_scoring.loc[credit_scoring['children'] == 20,'children']= 2

In [11]:
# Same category name is represented in 2 different ways, needs to be addressed
credit_scoring['education'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [12]:
# To identify whether there are more duplicate values, the education column was standardized
credit_scoring['education'] = credit_scoring['education'].str.lower()

In [13]:
credit_scoring.duplicated().sum()

71

In [14]:
# dropping duplicate values 
credit_scoring.drop_duplicates(keep=False,inplace=True) 


In [15]:
#Converting int 64 to int 32
credit_scoring = credit_scoring.astype({col: 'int32' for col in credit_scoring.select_dtypes('int64').columns})

# int64 columns have been converted to int32
print(credit_scoring.dtypes)

children                    int32
days_employed             float64
dob_years                   int32
education                  object
education_id                int32
family_status              object
family_status_id            int32
gender                     object
income_type                object
debt                        int32
total_income              float64
purpose                    object
days_employed_positive    float64
dtype: object


In [16]:
credit_scoring['total_income'] = credit_scoring['total_income'] .astype(int)

In [17]:
credit_scoring['days_employed_positive']= credit_scoring['days_employed_positive'] .astype(int)

In [18]:
credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21388 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   children                21388 non-null  int32  
 1   days_employed           19351 non-null  float64
 2   dob_years               21388 non-null  int32  
 3   education               21388 non-null  object 
 4   education_id            21388 non-null  int32  
 5   family_status           21388 non-null  object 
 6   family_status_id        21388 non-null  int32  
 7   gender                  21388 non-null  object 
 8   income_type             21388 non-null  object 
 9   debt                    21388 non-null  int32  
 10  total_income            21388 non-null  int32  
 11  purpose                 21388 non-null  object 
 12  days_employed_positive  21388 non-null  int32  
dtypes: float64(1), int32(7), object(5)
memory usage: 1.7+ MB


### Categorizing Data

In [19]:
#Renaming the columns names to give clear idea what is the data in the columns
credit_scoring.set_axis(['children_total','days_employed', 'age_years', 'education_level', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'default_status', 'total_income',  'loan_purpose', 'days_employed_final'], axis = 'columns', inplace = True)


4 categories created: 

    1. Kids and no kids
    2. Married + living together and single
    3. Income groups
    4. Loan purpose
   

    

In [20]:
def children_category(children_total):
    
    """
    The function divides the data into 2 categories
    1. People who have kids
    2. People who have no kids 
    """
    
    if children_total > 0:
        return 'have children'
    else :
        return 'no children'
credit_scoring['children_total_category'] = credit_scoring['children_total'].apply(children_category)

In [21]:
credit_scoring['family_status'].unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

In [22]:
def marital_status(family_status):
 
    """
    The function divides the data into 2 categories
    1. People living with somone

    2. People living alone
    """
    if family_status == 'married':
        return 'not single'
    if family_status == 'civil partnership':
        return 'not single'
    if family_status == 'widow / widower':
        return 'widow'
    if family_status == 'divorced':
        return 'single' 
    if family_status == 'unmarried':
        return 'single'
credit_scoring['family_status_category'] = credit_scoring['family_status'].apply(marital_status)

In [23]:
#Categorising the income group into 4 groups : low , low middle, high middle and high based on the total monthly income
def income_group(total_income):
    """
    The function returns the income group according to the total_income, using the following rules:
    —'low ' for total_income <= 17000
    —'low_middle ' for 17000< age <= 23000
     —'high_middle' for 23000< age <= 32000
    —'high' for all other cases
    """
    
    if total_income <= 17000:
        return 'low'
    if total_income <= 23000:
        return 'low middle'
    if total_income <= 32000:
        return 'high middle'
    if total_income > 32000:
        return 'high'
    

credit_scoring['total_income_category'] = credit_scoring['total_income'].apply(income_group)

In [24]:
# categorising the purpose for loan into 4 categories: Automobile, Education, House/Real Estate, Wedding
def purpose_category(purpose):
    if "car" in purpose:
        return 'Automobile'
    if "educa" in purpose:
        return 'Education'
    if "universit" in purpose:
        return 'Education'
    if "propert" in purpose:
        return 'House/Real Estate'
    if "estat" in purpose:
        return 'House/Real Estate'
    if "hous" in purpose:
        return 'House/Real Estate'
    if "weddi" in purpose:
        return 'Wedding'
credit_scoring['loan_purpose_1'] = credit_scoring['loan_purpose'].apply(purpose_category)
#credit_scoring

### Step 3. Identify which factors impact the customers ability to repay the loan using historical data which in turn could be used to create a credit scoring of a potential customer

In [25]:
credit_scoring_pivot = credit_scoring.pivot_table(index= ['children_total_category'], columns='default_status',values='total_income', aggfunc='count')
credit_scoring_pivot


default_status,0,1
children_total_category,,
have children,6672,678
no children,12975,1063


In [26]:
credit_scoring.groupby('children_total_category')['default_status'].mean()*100


children_total_category
have children    9.224490
no children      7.572304
Name: default_status, dtype: float64

In [27]:
credit_scoring_pivot = credit_scoring.pivot_table(index= ['children_total'], columns='default_status',values='total_income', aggfunc='count', fill_value = 0)
credit_scoring_pivot

default_status,0,1
children_total,,
0,12975,1063
1,4400,445
2,1923,202
3,303,27
4,37,4
5,9,0


In [28]:
credit_scoring.groupby('children_total')['default_status'].mean()*100

children_total
0    7.572304
1    9.184727
2    9.505882
3    8.181818
4    9.756098
5    0.000000
Name: default_status, dtype: float64

In [29]:
credit_scoring.groupby('family_status_category')['default_status'].mean()*100

family_status_category
not single    8.028975
single        8.970515
widow         6.576200
Name: default_status, dtype: float64

In [30]:
credit_scoring_pivot = credit_scoring.pivot_table(index= ['family_status'], columns='default_status',values='total_income', aggfunc='count', fill_value = 0)
credit_scoring_pivot

default_status,0,1
family_status,,
civil partnership,3740,388
divorced,1110,85
married,11369,931
unmarried,2533,274
widow / widower,895,63


In [31]:
credit_scoring.groupby('family_status')['default_status'].mean()*100

family_status
civil partnership    9.399225
divorced             7.112971
married              7.569106
unmarried            9.761311
widow / widower      6.576200
Name: default_status, dtype: float64

In [32]:
credit_scoring_pivot = credit_scoring.pivot_table(index= 'total_income_category', columns='default_status',values='income_type', aggfunc='count')
credit_scoring_pivot

default_status,0,1
total_income_category,,
high,4707,358
high middle,4946,458
low,4809,409
low middle,5185,516


In [33]:
credit_scoring.groupby('total_income')['default_status'].mean()*100

total_income
3306      100.0
3392        0.0
3418        0.0
3471        0.0
3503        0.0
          ...  
273809      0.0
274402      0.0
276204      0.0
352136    100.0
362496      0.0
Name: default_status, Length: 15389, dtype: float64

In [34]:
credit_scoring.groupby('total_income_category')['default_status'].mean()*100

total_income_category
high           7.068115
high middle    8.475204
low            7.838252
low middle     9.051044
Name: default_status, dtype: float64

In [35]:
credit_scoring_pivot = credit_scoring.pivot_table(index= 'loan_purpose_1', columns='default_status',values='total_income', aggfunc='count')
credit_scoring_pivot 

default_status,0,1
loan_purpose_1,,
Automobile,3894,403
Education,3634,370
House/Real Estate,10002,782
Wedding,2117,186


In [36]:
credit_scoring_pivot = credit_scoring.pivot_table(index= ['family_status_category', 'loan_purpose_1'],  columns='default_status',values='total_income', aggfunc='count')
credit_scoring_pivot 

default_status                               0    1
family_status_category loan_purpose_1              
not single             Automobile         2882  280
                       Education          2715  276
                       House/Real Estate  7395  577
                       Wedding            2117  186
single                 Automobile          814  103
                       Education           735   79
                       House/Real Estate  2094  177
widow                  Automobile          198   20
                       Education           184   15
                       House/Real Estate   513   28

In [37]:
credit_scoring.groupby('loan_purpose_1')['default_status'].mean()*100

loan_purpose_1
Automobile           9.378636
Education            9.240759
House/Real Estate    7.251484
Wedding              8.076422
Name: default_status, dtype: float64

### Findings of the factors : People with fewer kids and are married are more likely to payout the loan

1. 77% people in the data set had a marital status as married.
2. 66% of the total people in the data set had no kids
3. People with no kids had lower defaulter contribution than 8.1% , which is the defaulting population contribution to the total. 
4.  Then when we observe the the default chances based on the marital status, it is observed that data is skewed towards married people, with 77% of the people being in some form of partnership. However when we check the chances of defaulting, it was observed that single people (9%) tend to default higher than single.
5. The data was categorized into 4 homogeonous groups of income level, and it was found that people with the higher and lower income levels were less likely to default on the loan repayment as against those with middle level income level
6. 50% of the population wanted a loan for building a property. From past records it can be observed that people defaulted on repayments for automobile and education loan. However it is not clear whether the same people are looking to take the same loan.

### Step 4. Final conclusion

1. The data was a set of records for 21525 people.
2. Of which the data was missing for 4348 records. 
3. Each record provided details about a person's age, marital status, employment status, income level, whether they have had a loan before and whether they have defaulted earlier. The data also tells us what is the loan that the person is applying for. Apart from that the data contains details about the number of children the person has. 
4. Based on the data we were required to answer three questions what is the corelation between the chances of defaulting -
    a) Number of children
    b) marital status
    c) income level
    d) loan purpose
5. To prepare the data , the data was cleaned by removing the erroneous values, the missing values, and the duplicated data.
6. The first step was changing negative values in days_employment to positive values and correcting the very large values. 
7. Next we addressed the missing values in total income column. It was replaced by grouping the data by income type
8. We examined the 0 values in the age column, however did not replace it since these might have not been filled out by the customers to begin with 
9. The errorneous values in number of children was addressed.
10. Duplicates were dropped after processing the data. 